In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3'
os.environ['KMP_BLOCKTIME'] = '1'
import numpy as np
from dataset import get_labelled, get_unlabelled, _preload, _unload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [2]:
CONTEXT_SIZE = 16
BAG_SIZE = 256
ENCODER = 0
OUTPUT_SIZE = 256

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload(max_commit_bag_size = 8192, max_commits=4096)

Loading Commit lookup table


  0%|          | 0/70 [00:00<?, ?it/s]

Loading file ../data/commit_lookups/01_priority_commit_lookups.pickle


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Positive X_train:   0%|          | 0/9628 [00:00<?, ?it/s]

Generating Positive y_train:   0%|          | 0/9628 [00:00<?, ?it/s]

Generating Negative X_train:   0%|          | 0/38512 [00:00<?, ?it/s]

Generating Negative y_train:   0%|          | 0/38512 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Unsupervised X_train:   0%|          | 0/4096 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 1528
Train set split 0.1950261780104712
Test set size 382
Test set split 0.20418848167539266
Unsupervised Train Size 4096


In [ ]:
#For memory optimization
_unload()

In [8]:
model = CommitDiffModel(unsupervised_data_size = len(X_train_unsupervised))
model.initialize(encoder=ENCODER)

In [9]:
model.fit_siam(np.array(X_train_unsupervised), epochs=8, verbose=1)

Epoch 1/8
256/256 [==============================] - 30s 117ms/step - loss: -0.0263
Epoch 2/8
256/256 [==============================] - 33s 128ms/step - loss: -0.0553
Epoch 3/8
256/256 [==============================] - 30s 118ms/step - loss: -0.0858
Epoch 4/8
256/256 [==============================] - 30s 116ms/step - loss: -0.1132
Epoch 5/8
256/256 [==============================] - 30s 115ms/step - loss: -0.1384
Epoch 6/8
256/256 [==============================] - 29s 112ms/step - loss: -0.1618
Epoch 7/8
256/256 [==============================] - 29s 113ms/step - loss: -0.1834
Epoch 8/8
256/256 [==============================] - 29s 112ms/step - loss: -0.2033


In [10]:
model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=64, verbose=1)

Epoch 1/8
24/24 [==============================] - 4s 28ms/step - loss: 0.6148
Epoch 2/8
24/24 [==============================] - 1s 27ms/step - loss: 0.4957
Epoch 3/8
24/24 [==============================] - 1s 28ms/step - loss: 0.4913
Epoch 4/8
24/24 [==============================] - 1s 28ms/step - loss: 0.4902
Epoch 5/8
24/24 [==============================] - 1s 27ms/step - loss: 0.4929
Epoch 6/8
24/24 [==============================] - 1s 28ms/step - loss: 0.4992
Epoch 7/8
24/24 [==============================] - 1s 28ms/step - loss: 0.4919
Epoch 8/8
24/24 [==============================] - 1s 27ms/step - loss: 0.4838


In [11]:
model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)

0.5050214529037476